# Практическое задание 3 

# Классификация предложений с использованием BERT

## курс "Математические методы анализа текстов"


### ФИО: <впишите>

## Введение

### Постановка задачи

В этом задании вы будете классифицировать предложения из медицинских статей на несколько классов (background, objective и т.д.). 
Для того, чтобы улучшить качество решения вам предлагается дообучить предобученную нейросетевую архитектуру BERT.

### Библиотеки

Для этого задания вам понадобятся следующие библиотеки:
 - [Pytorch](https://pytorch.org/).
 - [Transformers](https://github.com/huggingface/transformers).
 
### Данные

Скачать данные можно здесь: [ссылка на google диск](https://drive.google.com/file/d/13HlWH8jnmsxqDKrEptxOXQg9kkuQMmGq/view?usp=sharing)

## Часть 1. Подготовка данных

Мы будем работать с предложениями из медицинских статей, разбитых на несколько классов. 

In [ ]:
import re
from collections import Counter

Путь к папке с данными:

In [ ]:
DATA_PATH = "sentence_classification_data"

Функция считывания данных:

In [ ]:
def read_data(file_name):
    """
    Parameters
    ----------
    file_name : str
        Pubmed sentences file path
        
    Returns
    -------
    text_data : list of str
        List of sentences for algorithm
    
    target_data : list of str
        List of sentence categories
    """
    text_data = []
    target_data = []

    with open(file_name, 'r') as f_input:
        for line in f_input:
            if line.startswith('#') or line == '\n':
                continue
            target, text = line.split('\t')[:2]    

            text_data.append(text)
            target_data.append(target)
    
    return text_data, target_data

Считывание данных:

In [ ]:
train_data, train_target = read_data(f'{DATA_PATH}/data_train.txt')
test_data, test_target = read_data(f'{DATA_PATH}/data_test.txt')
dev_data, dev_target = read_data(f'{DATA_PATH}/data_dev.txt')

## Часть 2. Построение бейзлайна (1 балл)

В этой части задания вам необходимо построить бейзлайн модель, с которой вы будете сравнивать ваше решение. В качестве бейзлайна вам предлагается использовать модель логистической регрессии на tf-idf представлениях.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

Перед тем как подать в модель предложения, необходимо их предобработать:
    
1. привести все предложения к нижнему регистру
2. удалить из предложений все непробельные символы кроме букв, цифр
3. все цифры заменить на нули

Метки ответов необходимо преобразовать из текстового вида в числовой (это можно сделать с помощью LabelEncoder).

Затем необходимо построить tf-idf матрицу по выбранным предложениям (используйте для подсчёта tf-idf только train_data!) и обучить на них модель логистической регрессии. Используйте dev выборку для подбора гиперпараметров модели. Добейтесь того, что на test и dev выборках accuracy будет будет выше 0.8.

In [ ]:
#### your code here ####

## Часть 3. Задание BERT (4 балла за 3 и 4 части)

Так как обучающих предложений очень мало, попробуем использовать модель BERT, предобученную на большом датасете. Будем использовать библиотеку transformers. Для обучения модели используйте данные до обработки из предыдущего пункта.

In [ ]:
BERT_MODEL_NAME = "bert-base-uncased"
NUM_LABELS = len(set(y_train))

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AdamW, WarmupLinearSchedule

import torch
from torch.utils.data import DataLoader, Dataset

Модель BERT работает с специальным форматом данных — все токены из предложения получены с помощью алгоритма BPE. Класс BertTokenizer позволяет получить BPE разбиение для предложения.

In [ ]:
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_NAME)

В библиотеке transformers есть специальный класс для работы с задачей классификации — BertForSequenceClassification. Воспользуемся им, чтобы задать модель.

In [ ]:
bert_model = BertForSequenceClassification.from_pretrained(
    BERT_MODEL_NAME, num_labels=NUM_LABELS
)

bert_model.to('cuda')

Реализуем специальный кастомный датасет для токенизированных с помощью BPE предложений. Каждое предложение должно быть преобразовано в последовательность BPE индексов. Не забудьте, что в начале каждого предложения должен стоять специальный токен [CLS], а в конце должен стоять специальный токен [SEP].

Задайте датасет, используя BertTokenizer:

In [ ]:
class BertTokenizedDataset(Dataset):
    def __init__(self, tokenizer, text_data, target_data=None, max_length=256):
        """
        Parameters
        ----------
        tokenizer : instance of BertTokenizer
        text_data : list of str
            List of input sentences
        target_data : list of int
            List of input targets
        max_length : int
            Maximum length of input sequence (length in bpe tokens)
        """
        #### your code here ####
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, i):
        if self.target_data is not None:
            return self.data[i], self.target_data[i]
        else:
            return self.data[i]

Получите все датасеты для всех типов данных. 

**Замечание**. После получения есть смысл сохранить все датасеты на диск, т.к. предобработка занимает время.

In [ ]:
train_dataset = BertTokenizedDataset(tokenizer, train_data, y_train)
dev_dataset = BertTokenizedDataset(tokenizer, dev_data, y_dev)
test_dataset = BertTokenizedDataset(tokenizer, test_data, y_test)

Используем  класс PadSequences, чтобы задать способ паддинга, работающий с встроенным в pytorch DataLoader.

In [ ]:
class PadSequences:
    def __init__(self, use_labels=False):
        self.use_labels = use_labels
    
    def __call__(self, batch):
        """
        Parameters
        ----------
        batch : list of objects or list of (object, label)
            Each object is list of int indexes.
            Each label is int.
        """
        data_label_batch = batch if self.use_labels else [(x, 0) for x in batch]
            
        # Sort the batch in the descending order
        sorted_batch = sorted(data_label_batch, key=lambda x: x[0].shape[0], reverse=True)
        # Get each sequence and pad it
        sequences = [x[0] for x in sorted_batch]
        sequences_padded = torch.nn.utils.rnn.pad_sequence(sequences, batch_first=True)
        max_lenght = len(sequences[0])

        # Also need to store the length of each sequence
        # This is later needed in order to unpad the sequences
        lengths = torch.LongTensor([[1] * len(x) + [0] * (max_lenght - len(x)) for x in sequences])
        # Don't forget to grab the labels of the *sorted* batch
        
        if self.use_labels:
            labels = torch.LongTensor([x[1] for x in sorted_batch])
            return sequences_padded, lengths, labels
        else:
            return sequences_padded

Зададим DataLoader для каждого из датасетов:

In [ ]:
BATCH_SIZE = 16

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,
                              collate_fn=PadSequences(use_labels=True))

dev_dataloader = DataLoader(dev_dataset, batch_size=BATCH_SIZE, shuffle=False,
                              collate_fn=PadSequences(use_labels=True))

test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False,
                              collate_fn=PadSequences(use_labels=True))

Заметьте, что модель трансформера обучается по достаточному большому размеру батча (обычно 64), который скорее всего не будет влезать на вашу видеокарту. Поэтому, рекомендуется "накапливать" градиенты за несколько итераций. С помощью параметра ACCUMULATION_STEPS задайте, раз в сколько итераций вам необходимо делать шаг метода оптимизации.

In [ ]:
EPOCH_AMOUNT = 2
TRAIN_LENGTH = len(train_dataset)
BATCH_SIZE = 16
ACCUMULATION_STEPS = 4

LR = 2e-5

Посчитайте общее число раз, когда ваш оптимизатор будет делать обновления на основе выбранных значений EPOCH_AMOUNT, BATCH_SIZE, ACCUMULATION_STEPS и  TRAIN_LENGTH. Эта величина нужна для правильного задания параметров оптимизаторов.

In [ ]:
train_optimization_step_amount = ### your code here ####

Зададим параметры оптимизаторов. Мы будем использовать специальные оптимизаторы из библиотеки transformers AdamW и WarmupLinearSchedule, обеспечивающие плавный разгон и медленное затухание темпа обучения.

In [ ]:
optimizer = AdamW(bert_model.parameters(), lr=LR, correct_bias=False)
scheduler = WarmupLinearSchedule(
    optimizer,
    warmup_steps=train_optimization_step_amount * 0.05,
    t_total=train_optimization_step_amount,
)

Для некоторых групп параметров зададим коэффициенты регуляризации.

In [ ]:
param_optimizer = list(bert_model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

## Часть 4. Обучение BERT 

Теперь всё готово к тому, чтобы дообучить BERT на датасете train_dataset!

Используйте dev_dataset для выбора гиперпараметров модели и обучения. Задание будет засчтано на полный балл если на dev_dataset и test_dataset точность будет выше 0.84.

In [ ]:
#### your code here ####

## Бонусная часть (до 3 баллов)

Улучшите качество (на обеих выборках), используя любые способы (кроме использования дополнительных обучающих данных датасета RCT2000):

* $> 0.86$ — 1 балл 
* $> 0.88$ — 2 балла
* $> 0.9$ — 3 балла